### Tensor Flow 2-11 notebook

Notes: 
- Notebook should be running with an Nvidia GPU to for top performance
- load_img will load a nparray with x, y, 3 color dims.  
- If we're just processing one image we will need to np.expand_dims(image, axies=0) to get 1, x, y, 3. The library expects N images in the first dim
- should set random seed on tf.random.set_seed(1) for reproduceability 
- 

last change 3/6/2025

In [1]:
# import the libraries for training, testing, validation
import tensorflow as tf
from tensorflow.keras.applications import imagenet_utils  # will decode predictions out of the model into a 4 dim array of N (image num), imageID, label, probability result[0] would be the set of results for image one
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img  # will load img and reshape, usage is load_img(image_name_loc, target_size=input_shape)
from tensorflow.keras.utils import plot_model  # Note: usage syntax is plot_model(instantied_model, to_file='', show_shapes=True)

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds  # For loading datasets from GCS
import os


2025-03-08 13:05:01.055102: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-08 13:05:04.138005: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-03-08 13:05:04.138244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local

In [2]:
# training packages
# import the necessary packages
# from tensorflow import reduce_mean
# from tensorflow.keras import Model
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Conv2D
# from tensorflow.keras.layers import InputLayer
# from tensorflow.keras.layers import Conv2DTranspose
# from tensorflow.keras.losses import Reduction
# from tensorflow.keras.losses import MeanSquaredError
# from tensorflow.image import convert_image_dtype
# from tensorflow.keras.preprocessing.image import array_to_img

# import tensorflow_datasets as tfds

In [3]:
# Configuration
GCS_BUCKET = 'nabirds_filtered'  
DATASET_PATH = 'images'  # Relative path within the bucket
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = None  # Will be determined from the dataset

In [4]:
def load_gcs_dataset(bucket_name, dataset_path, image_size, batch_size):
    """Loads a dataset from Google Cloud Storage."""

    gcs_dataset_path = f"gs://{bucket_name}/{dataset_path}"

    try:
        # Attempt to load the dataset using tf.keras.utils.image_dataset_from_directory.
        # This handles directory structures well.

        dataset = keras.utils.image_dataset_from_directory(
            gcs_dataset_path,
            image_size=image_size,
            batch_size=batch_size,
            label_mode='categorical', # important for softmax
        )
        return dataset
    except Exception as e:
        print(f"Error loading dataset from GCS: {e}")
        return None


In [5]:
train_dataset = load_gcs_dataset(GCS_BUCKET, os.path.join(DATASET_PATH, 'train'), IMAGE_SIZE, BATCH_SIZE)
validation_dataset = load_gcs_dataset(GCS_BUCKET, os.path.join(DATASET_PATH, 'validation'), IMAGE_SIZE, BATCH_SIZE)

if train_dataset is None or validation_dataset is None:
    print("Dataset loading failed. Exiting.")
    exit()

NUM_CLASSES = len(train_dataset.class_names) # Get class count.


Error loading dataset from GCS: Could not find directory gs://nabirds_filtered/images/train
Error loading dataset from GCS: Could not find directory gs://nabirds_filtered/images/validation
Dataset loading failed. Exiting.


AttributeError: 'NoneType' object has no attribute 'class_names'

In [ ]:

# --- 2. Load MobileNetV2 and Modify Head ---
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unlock the base model (you can choose to freeze some layers if needed)
base_model.trainable = True

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Optional: Add more dense layers
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# --- 3. Compile the Model ---
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',  # Important for multi-class classification
              metrics=['accuracy'])

# --- 4. Train the Model ---
epochs = 10  # Adjust as needed

history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset
)

# --- 5. Save the Model (Optional) ---
model.save('mobilenet_retrained.h5')

print("Training complete.")